In [1]:
from facetorch import FaceAnalyzer
from facetorch.analyzer.utilizer import LandmarkDrawerTorch
from omegaconf import OmegaConf
from torch.nn.functional import cosine_similarity
from typing import Dict
import operator
import torchvision
import os
import shutil
import torch
import numpy as np
import pandas as pd


In [2]:
DATASET_PATH = './Curated_Dataset/'
ANCHOR_PATH = './dataset/anchor_points_dataset/'
FACE_PATH = './dataset/face_dataset/'

if os.path.exists(FACE_PATH):
    shutil.rmtree(FACE_PATH)

if os.path.exists(ANCHOR_PATH):
    shutil.rmtree(ANCHOR_PATH)

os.makedirs(FACE_PATH)
os.makedirs(ANCHOR_PATH)

cfg = OmegaConf.load("./gpu.config.yml")


In [3]:

from scipy.spatial.transform import Rotation as R

def Rotate_Face(pts):
    data = pts[0]

    x = data[0, :]
    y = data[1, :]
    z = data[2, :]

    face_tensor = torch.stack([x, y, z], dim=1).to('cuda:0')
    # Väldigt hustler-lösning, får inte skiten att se bra ut

    sum_vectors = torch.sum(face_tensor, dim=0)
    centroid = sum_vectors / face_tensor.shape[0]
    face_tensor_orgin = (face_tensor - centroid).cpu().numpy()


    P1 = face_tensor_orgin[0]
    P2 = face_tensor_orgin[16]

    y1 = P1[1]
    y2 = P2[1]

    while abs(y1 - y2) > 1:
        r = R.from_euler('z', -0.1, degrees=True)
        face_tensor_orgin =  face_tensor_orgin @ r.as_matrix().T
        P1 = face_tensor_orgin[0]
        P2 = face_tensor_orgin[16]
        y1 = P1[1]
        y2 = P2[1]



    ### VET INTE OM DETTA FUNKAR ###
    z1 = P1[2]
    z2 = P2[2]
    x = face_tensor_orgin[33][0]

    i = 0.1
    if x > 0:
        i = -0.1

    while abs(z1 - z2) > 1:
        r = R.from_euler('y', i, degrees=True)
        face_tensor_orgin =  face_tensor_orgin @ r.as_matrix().T
        P1 = face_tensor_orgin[0]
        P2 = face_tensor_orgin[16]
        z1 = P1[2]
        z2 = P2[2]

    ### VET INTE OM DETTA FUNKAR SLUT ###

    if face_tensor_orgin[25][1] > 0:
        r = R.from_euler('z', 180, degrees=True)
        face_tensor_orgin =  face_tensor_orgin @ r.as_matrix().T
    
    return face_tensor_orgin

def fft_func(arr):
    norm = np.linalg.norm(arr)  # Calculate the L2 norm of the data
    normalized_data = arr / norm
    transformed = np.fft.fftn(normalized_data)

    magnintude = np.abs(transformed)

    transformed_array = np.zeros(shape=(magnintude.shape[0]-1, magnintude.shape[1]))
    for i in range(0,3):
        background_value = magnintude[:,i][0]
        column = magnintude[:,i][1:]

        column = column / background_value

        transformed_array[:,i] = column


    return transformed_array

def Process_Image(path_img):
    """
    Processes the input image and extracts important features (anchor points).

    Args:
        path_to_image (str): The file path to the image to be processed.

    Returns:
        processed_image: The image after processing 
        anchor_points: A set of key points or features detected in the image (e.g., corners, edges, or other important features).
    """
    analyzer = FaceAnalyzer(cfg.analyzer)
    response = analyzer.run(
        path_image=path_img,
        batch_size=cfg.batch_size,
        return_img_data=cfg.return_img_data,
        include_tensors=cfg.include_tensors,
        fix_img_size=cfg.fix_img_size
    )
    return response

def Save_Processed_Image(response, path):
    pil_image = torchvision.transforms.functional.to_pil_image(response.img)
    pil_image.save(path)
    return 

def Save_Processed_Feature_Vector(arr, name, data):
    arr = Rotate_Face(arr)
    fft = fft_func(arr)
    flat_list = fft.reshape(-1)
    data.loc[len(data)] = flat_list.tolist() + [name]

    return


In [12]:
columns = [f'feature_{i}' for i in range(1, 202)] + ['label']
df = pd.DataFrame(columns=columns)
for name in os.listdir(DATASET_PATH):

    temp_path = DATASET_PATH + f'{name}'  
    temp_anchor_path = ANCHOR_PATH + f'{name}'
    temp_face_path = FACE_PATH + f'{name}'

    if os.path.exists(temp_face_path):
        shutil.rmtree(temp_face_path)


    os.makedirs(temp_face_path)
    i = 0
    for image in os.listdir(temp_path):

        response = Process_Image( temp_path + f'/{image}')
        pts = [face.preds["align"].other["lmk3d"].cpu() for face in response.faces]
        if len(pts) == 1:
            
            Save_Processed_Image(response, f'{temp_face_path}/{name}{i}.jpg')
            Save_Processed_Feature_Vector(pts, name, df)
            i += 1
    
    if os.path.isdir(temp_face_path) and len(os.listdir(temp_face_path)) == 0:
        os.rmdir(temp_face_path)
    
    df.to_csv(ANCHOR_PATH + 'processed_data.csv', index=False)
    







{"asctime": "2024-10-18 13:52:41,132", "levelname": "INFO", "message": "Initializing FaceAnalyzer"}
{"asctime": "2024-10-18 13:52:41,133", "levelname": "INFO", "message": "Initializing BaseReader"}
{"asctime": "2024-10-18 13:52:41,570", "levelname": "INFO", "message": "Initializing FaceDetector"}
{"asctime": "2024-10-18 13:52:41,992", "levelname": "INFO", "message": "Initializing FaceUnifier"}
{"asctime": "2024-10-18 13:52:42,004", "levelname": "INFO", "message": "Initializing FacePredictor objects"}
{"asctime": "2024-10-18 13:52:42,005", "levelname": "INFO", "message": "Initializing FacePredictor embed"}
{"asctime": "2024-10-18 13:52:42,214", "levelname": "INFO", "message": "Initializing FacePredictor verify"}
{"asctime": "2024-10-18 13:52:43,385", "levelname": "INFO", "message": "Initializing FacePredictor au"}
{"asctime": "2024-10-18 13:52:44,035", "levelname": "INFO", "message": "Initializing FacePredictor va"}
{"asctime": "2024-10-18 13:52:44,086", "levelname": "INFO", "message": 